# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f00a4320f40>
├── 'a' --> tensor([[ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210]])
└── 'x' --> <FastTreeValue 0x7f00a4320ee0>
    └── 'c' --> tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                        [ 0.0972,  0.4060,  0.4272,  0.5060],
                        [-0.3488, -0.5697,  2.5417,  1.2365]])

In [4]:
t.a

tensor([[ 1.0102, -0.3488, -1.3711],
        [-1.0117,  0.5725,  0.8210]])

In [5]:
%timeit t.a

65.1 ns ± 0.0436 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f00a4320f40>
├── 'a' --> tensor([[-2.2053,  1.5851, -0.0245],
│                   [-1.1834,  1.3241,  0.1968]])
└── 'x' --> <FastTreeValue 0x7f00a4320ee0>
    └── 'c' --> tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                        [ 0.0972,  0.4060,  0.4272,  0.5060],
                        [-0.3488, -0.5697,  2.5417,  1.2365]])

In [7]:
%timeit t.a = new_value

69.2 ns ± 0.0789 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210]]),
    x: Batch(
           c: tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                      [ 0.0972,  0.4060,  0.4272,  0.5060],
                      [-0.3488, -0.5697,  2.5417,  1.2365]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0102, -0.3488, -1.3711],
        [-1.0117,  0.5725,  0.8210]])

In [11]:
%timeit b.a

58.8 ns ± 0.0512 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4012, -1.4102,  0.9832],
               [-0.0778,  1.6890, -0.2754]]),
    x: Batch(
           c: tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                      [ 0.0972,  0.4060,  0.4272,  0.5060],
                      [-0.3488, -0.5697,  2.5417,  1.2365]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.161 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

834 ns ± 1.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.6 µs ± 9.88 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 370 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7effeecda670>
├── 'a' --> tensor([[[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]],
│           
│                   [[ 1.0102, -0.3488, -1.3711],
│                    [-1.0117,  0.5725,  0.8210]]])
└── 'x' --> <FastTreeValue 0x7effee63b9a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.9 µs ± 46.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7effee5ecb20>
├── 'a' --> tensor([[ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210],
│                   [ 1.0102, -0.3488, -1.3711],
│                   [-1.0117,  0.5725,  0.8210]])
└── 'x' --> <FastTreeValue 0x7effee5ecfa0>
    └── 'c' --> tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                        [ 0.0972,  0.4060,  0.4272,  0.5060],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.8 µs ± 71 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]],
       
               [[ 1.0102, -0.3488, -1.3711],
                [-1.0117,  0.5725,  0.8210]]]),
    x: Batch(
           c: tensor([[[ 2.5481, -0.6697, -1.4984,  0.2766],
                       [ 0.0972,  0.4060,  0.4272,  0.5060],
                       [-0.3488, -0.5697,  2.5417,  1.2365]],
         

In [26]:
%timeit Batch.stack(batches)

77.4 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210],
               [ 1.0102, -0.3488, -1.3711],
               [-1.0117,  0.5725,  0.8210]]),
    x: Batch(
           c: tensor([[ 2.5481, -0.6697, -1.4984,  0.2766],
                      [ 0.0972,  0.4060,  0.4272,  0.5060],
                      [-0.3488, -0.5697,  2.5417,  1.2365],
                      [ 2.5481, -0.6697, -1.4984,  0.2766],
                      [ 0.0972,  

In [28]:
%timeit Batch.cat(batches)

165 µs ± 573 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

328 µs ± 798 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
